In [36]:
import os
import requests, sys, webbrowser, bs4
import pymongo
import pickle
import random

In [25]:
errors = ['Kevin_Cavanaugh_(New_Hampshire)']

client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
database = client['legislative_documents']
collection = database['legiscan_people_data']


positions = ["vice chair", "clerk", "chair"] # other position is member
root = "https://ballotpedia.org/"
committee_table = {}
politicians = []
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')

end_url = "_(New_Hampshire)"
for people in collection.find():
    try:

        p = people['ballotpedia']

        end_url = "_(New_Hampshire)"

        committee_table[p] = {'year':[], 'committee/positions': []}


        url = root + p
        res = requests.get(url)
        res.raise_for_status()

        soup = bs4.BeautifulSoup(res.text, features="html.parser")
        check_multi_name = soup.find('table', id="disambig")

        if not check_multi_name is None:
            new_url = url + end_url
            new_res = requests.get(new_url)
            new_res.raise_for_status()

            soupy = bs4.BeautifulSoup(new_res.text, features="html.parser")


            sessions = soupy.find_all('table')

            for j in range(len(sessions)):

                val = sessions[j].find('b')
                try:
                    stringy = val.text
                    res = stringy.lower()
                    if "committee" in res:
                        _time = int(res[-4:])
                        if _time < 2015:
                            temp_val = _time + 1
                            time = str(_time) + "-" + str(temp_val)
                        else:
                            time = str(_time)

                        tempy = sessions[j].find_all('td')
                        for t in tempy:
                            _ret = ''.join(filter(whitelist.__contains__, t.text))
                            ret = _ret.lower()

                            committee_table[p]['year'].append(time)
                            insert_mask = 0
                            for pos in positions:
                                if pos in ret:
                                    committee = ret.replace(pos, "")
                                    result = committee + "/" + pos
                                    committee_table[p]['committee/positions'].append(result)
                                    insert_mask = 1
                                    break
                            if insert_mask == 0:
                                committee = ret
                                result = committee + "/" + "member"
                                committee_table[p]['committee/positions'].append(result)

                except:
                    pass

        else:
            sessions = soup.find_all('table')

            for j in range(len(sessions)):

                val = sessions[j].find('b')
                try:
                    stringy = val.text
                    res = stringy.lower()
                    if "committee" in res:

                        _time = int(res[-4:])

                        if _time < 2015:
                            temp_val = _time + 1
                            time = str(_time) + "-" + str(temp_val)
                        else:
                            time = str(_time)
                        tempy = sessions[j].find_all('td')
                        for t in tempy:
                            _ret = ''.join(filter(whitelist.__contains__, t.text))
                            ret = _ret.lower()

                            committee_table[p]['year'].append(time)
                            insert_mask = 0
                            for pos in positions:
                                if pos in ret:
                                    committee = ret.replace(pos, "")
                                    result = committee + "/" + pos
                                    committee_table[p]['committee/positions'].append(result)
                                    insert_mask = 1
                                    break
                            if insert_mask == 0:
                                committee = ret
                                result = committee + "/" + "member"
                                committee_table[p]['committee/positions'].append(result)

                except:
                    pass
    except:
        pass
with open('/home/ec2-user/SageMaker/ProjectQuigley/sponsor_comittee_table.dict', 'wb') as f:
    pickle.dump(committee_table, f)



In [28]:
with open('/home/ec2-user/SageMaker/ProjectQuigley/sponsor_comittee_table.dict', 'wb') as f:
    pickle.dump(committee_table, f)

In [30]:
print(committee_table['David_Lundgren'])
print(len(committee_table))

{'year': ['2017', '2015', '2013-2014', '2011-2012'], 'committee/positions': [' statefederal relations and veterans affairs/member', ' statefederal relations and veterans affairs/member', ' statefederal relations and veterans affairs/member', ' statefederal relations and veterans affairs/member']}
1735


In [39]:
print(len(committee_table))
empty = []
counter = 0
for i in committee_table.keys():
    if committee_table[i]['year'] == []:
        empty.append(i)
print(len(empty))

print(random.sample(empty, 50))

1735
1097
['Juan_Vargas', 'Dave_Loebsack', 'Mike_Kelly_(Pennsylvania)', 'Kevin_Cavanaugh_(New_Hampshire)', 'Ron_Kind', 'Karen_Bass', 'Barbara_Mikulski', 'Jane_Harman', 'Matt_Salmon', 'Ted_Yoho', 'David_Trott', 'Eric_Swalwell', 'Cathy_McMorris_Rodgers', 'Paul_Kanjorski', 'Kathleen_Rice', 'Brad_Miller', 'Randy_Weber', 'Brian_Babin', 'Deanna_Jurius', 'Joe_Garcia_(Florida)', 'Edolphus_Towns', 'Rand_Paul', 'Ron_Barber', 'Mary_Gay_Scanlon', 'Henry_Cuellar', 'Charlie_Melancon', 'Nan_Hayworth', 'Jo_Ann_Emerson', 'Spencer_Bachus', 'Renee_Ellmers', 'Timothy_Williams_(Texas)', 'Robert_Marshner', 'Ro_Khanna', 'Brian_Bilbray', 'Steve_Hobbs,_Missouri_Representative', 'Mark_Pocan', 'Candice_Miller', 'John_Hoeven', 'Richard_Shelby', 'Corrine_Brown', 'Pedro_Pierluisi', 'Josh_Gottheimer', 'Chris_Coons', 'Michael_Doyle_(Pennsylvania_Congress)', 'Susan_Davis', 'John_Culberson', 'Victoria_Schwaegler', 'Kevin_Pratt', 'Suzan_DelBene', 'Trey_Gowdy']


In [48]:
errors = []

client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
database = client['legislative_documents']
collection = database['legiscan_people_data']

for peeps in collection.find({'ballotpedia': {'$in': empty}}):
    if peeps['state_id'] == 29:
        errors.append(peeps['people_id'])
    

In [50]:
print(len(errors))
print(errors)
print(counter)

171
[16347, 16753, 17496, 17929, 18144, 18171, 19177, 19184, 19397, 19429, 19430, 19431, 19432, 19435, 19460, 19474, 19475, 19600, 19743, 19744, 19745, 19746, 19747, 19748, 19749, 19750, 19751, 19752, 19753, 19754, 19755, 19756, 19757, 19758, 19759, 19760, 19761, 19762, 19763, 19764, 19765, 19766, 19767, 19768, 19769, 19770, 19771, 19772, 19773, 19774, 19775, 19776, 19777, 19778, 19779, 19780, 19781, 19782, 19783, 19784, 19785, 19786, 19787, 19788, 19789, 19790, 19791, 19792, 19793, 19794, 19795, 19796, 19797, 19798, 19799, 19800, 19801, 19802, 19803, 19804, 19805, 19806, 19807, 19808, 19809, 19810, 19811, 19812, 19813, 19814, 19815, 19816, 19817, 19818, 19819, 19820, 19821, 19822, 19823, 19824, 19825, 19826, 19827, 19828, 19829, 19830, 19831, 19832, 19833, 19834, 19835, 19836, 19837, 19838, 19839, 19840, 19841, 19842, 19843, 19844, 19845, 19846, 19847, 19848, 19849, 19850, 19851, 19852, 19853, 19854, 19855, 19856, 19857, 19858, 19859, 19860, 19861, 19862, 19863, 19864, 19865, 19866, 1

In [47]:
#committee_table["David_Pierce_(New_Hampshire_Senate)"]['year'] =["2015", "2015", "2013-2014", "2013-2014",  "2011-2012", "2011-2012"]
#committee_table["David_Pierce_(New_Hampshire_Senate)"]['committee/positions'] =[" public works and highways/member", " public works and highways/member"]







KeyError: 'David_Pierce_(New_Hampshire_Senate)'